# Group 34: Predicting Subscription Status from Age and Play Time

In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
library(readr)
library(ggplot2)
library(dplyr)
library(RColorBrewer)
options(repr.matrix.max.rows = 6)

## Introduction

As more improvements and innovations have been made in different aspects of video games such as mode and style design, video games started to play significant roles in the daily lives of individuals. A research group in the Computer Science department at the University of British Columbia (UBC), led by Professor Frank Wood, has established a Minecraft research server to collect data on how people interact within the game environment (1). Specifically, this research group collected two datasets, the players.csv summarized data about player information, and the sessions.csv which summarizes playing sessions. Our report further analyzes the data collected by this research group to investigate a specific question: Can total played hours and age predict subscription in the players.csv dataset? It is expected that players with greater played hours and a younger age would tend to subscribe the game-related newsletter.
<br>
<br>
In the players.csv dataset, there are 196 observations with 7 variables, summarizing players' information on the Minecraft server. Numerical data contains played hours and player age, character data contains the level of experience on the game, hashed email (user identifier), name of the players, and gender of the players. Logical data contains the status of the subscription to a game-related newsletter.

## Methods

### Wrangling

In [ ]:
url_players <- "https://raw.githubusercontent.com/lucy-diaz/DSCIprojectgroup34/refs/heads/main/players.csv"
players <- read_csv(url_players)
players

In [ ]:
tidy_player<- players|>
        mutate(experience = factor(experience), gender = factor(gender),subscribe=factor(subscribe))
tidy_player

clean_player<- tidy_player|>
                select(subscribe,played_hours,Age)|>
                filter(played_hours!=0.0)|>
                filter(!is.na(Age))

clean_player


### Summary

We used Knn-Classification to test if the age and playtime of players can predict their subscription status. Firstly, we wrangled the data to keep only the 3 variables needed, subscribe, played_hours and Age. Secondly, we split the data into training and testing in a 75:25 split. Once this was completed, the K value was chosen through cross-validation in a 5 fold split and the analysis completed using K = 7. We trained the model using the training data and then applied it to the testing data, creating a confusion matrix to interpret the recall/precision of our model.

### Visialization of Data Set

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 11)
age_average_played_hours <- clean_player |> group_by(Age, subscribe)|> summarize(average_played_hours = mean(played_hours))|>
ggplot(aes(x= Age, y= average_played_hours, color = subscribe))+
geom_point()+
labs(x= "Age", y= "Average Played hours", title = "Figure 1: Average Played Hours vs Age", color = "Subscription Status")+
theme(text = element_text(size = 20))
age_average_played_hours

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 11)
age_observations <- clean_player |>
ggplot(aes(x= Age, fill = subscribe))+
geom_histogram(binwidth = 1.5)+
labs(x= "Age", y= "Number of Players", title = "Figure 2: Number of Players per Age and Subscription Status", fill = "Subscribed")+
theme(text = element_text(size = 20))
age_observations

### Analysis

In [ ]:

set.seed(123)

# Split the data into training (75%) and testing (25%) sets using initial_split()
split <- initial_split(clean_player, prop = 0.75, strata = subscribe)
train_data <- training(split)
test_data  <- testing(split)

# Create a recipe to center and scale the predictors (Age and played_hours)
rec <- recipe(subscribe ~ Age + played_hours, data = train_data) |>
  step_center(all_predictors()) |>
  step_scale(all_predictors())

# Specify a KNN model with neighbors to be tuned using nearest_neighbor()
knn_mod <- nearest_neighbor(neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

# Create a workflow combining the recipe and the KNN model
wf <- workflow() |>
  add_recipe(rec) |>
  add_model(knn_mod)

# Set up 5-fold cross-validation on the training data using vfold_cv()
cv_folds <- vfold_cv(train_data, v = 5, strata = subscribe)

# Create a grid of k values to test (from 1 to 10) using tibble()
grid_vals <- tibble(neighbors = 1:10)

# Tune the model using tune_grid() with the defined cross-validation folds and grid
tune_results <- tune_grid(wf, resamples = cv_folds, grid = grid_vals)

# Collect the metrics and select the best k based on accuracy
metrics <- collect_metrics(tune_results)
best_k <- metrics |>
  filter(.metric == "accuracy") |>
  arrange(desc(mean)) |>
  slice(1) |>
  pull(neighbors)
cat("Best k selected:", best_k, "\n")

# Finalize the KNN model using the best k
final_knn_mod <- nearest_neighbor(neighbors = best_k) |>
  set_engine("kknn") |>
  set_mode("classification")
final_wf <- workflow() |>
  add_recipe(rec) |>
  add_model(final_knn_mod)
final_fit <- fit(final_wf, data = train_data)

# Predict subscription status on the testing data
predictions <- predict(final_fit, new_data = test_data) |>
  bind_cols(test_data)
predictions


### Analysis Visualization 

In [ ]:
#Accuracy VS K plot
Acc_vs_k_plot<-collect_metrics(tune_results) |>
  filter(.metric == "accuracy") |>
  ggplot(aes(x = neighbors, y = mean)) +
  geom_line() +
  geom_point() +
  labs(
    title = "Model Accuracy vs. k",
    x = "Number of Neighbors (k)",
    y = "Accuracy"
  )

Acc_vs_k_plot

In [ ]:
# --- Basic Visualization ---
# Scatter plot of testing data: Age vs. played_hours colored by predicted subscription status
predictions |>
  ggplot(aes(x = Age, y = played_hours, color = .pred_class)) +
  geom_point(alpha = 0.7, size = 2) +
  labs(
    title = paste("KNN Prediction: Subscription Status (k =", best_k, ")"),
    x = "Player Age (scaled)",
    y = "Total Hours Played (scaled)",
    color = "Predicted Status")
 

predictions

In [ ]:
library(yardstick)

pred_matrix<- conf_mat(predictions, truth = subscribe, estimate = .pred_class) |>
  autoplot(type = "heatmap") +
  labs(title = "Confusion Matrix (KNN Predictions)")

pred_matrix

metrics<-predictions|>
    metrics(truth=subscribe, estimate=.pred_class)|>
    filter(.metric=="accuracy")

metrics
      

## Discussion

### Summary of Findings

### Hypothesis vs Results

The objective has been met, as our model can predict subscription with Age and Played hours with a pretty high accuracy of 78.6%. We hypothesized that players with greater played hours and a younger age would tend to subscribe to the game-related newsletter. According to KNN prediction: Subscription Status (k=7) graph, a dense cluster of the data points from age 13-25 indicated "TRUE" for the subscription, which supports how a younger age tends to subscribe more. However, a dense cluster of data points below 25 hours played time still shows mostly TRUE predictions for subscription, which does not support the hypothesis. Overall, the hypothesis is not supported.
### Impact

### Future Analysis

### Reference
(1)The University of British Columbia. (n.d.). Retrieved March 29, 2025, from https://canvas.ubc.ca/courses/153254/assignments/2055150?module_item_id=7644030
